In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import json
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import spacy
import Utility as util
from Utility import REDataset_entities, REModelWithAttention

In [2]:
df_fewRel = pd.read_csv('fewRel_entities.csv')

In [3]:
df_aug_llama = pd.read_csv('aug_llama_entities_cleaned.csv')

In [4]:
print(f'orignal dataset = {len(df_fewRel)} and augumented data = {len(df_aug_llama)}')

orignal dataset = 44800 and augumented data = 2783


In [6]:
df_aug_llama = util.sampleData(df_aug_llama,'relation')

In [5]:
len(df_aug_llama)

2783

In [6]:
train_df, test_df = train_test_split(df_fewRel, test_size=0.2, random_state=42) # To take test set from orignal data
print(f'Train set = {len(train_df)} and test set = {len(test_df)}')

Train set = 35840 and test set = 8960


In [7]:
df_combined = pd.concat([train_df, df_aug_llama], ignore_index=True) # combined augumented data with orignal FewRel training data
print(len(df_combined))

38623


In [8]:
# Encode training and test set for relation
label_encoder = LabelEncoder()
df_combined['relation_id'] = label_encoder.fit_transform(df_combined['relation'])
test_df['relation_id'] = label_encoder.fit_transform(test_df['relation'])

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # Choose the same model as in your model
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [10]:
max_seq_length = 128  
train_dataset = util.REDataset_entities(df_combined,tokenizer, max_seq_length)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = util.REDataset_entities(test_df, tokenizer, max_seq_length)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [11]:
model = REModelWithAttention(tokenizer, num_classes=len(label_encoder.classes_)).to(device)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [12]:
train_loss, valid_loss = util.train(model, train_loader, test_loader, criterion, optimizer, device, patience=4, num_epochs=20) # All with entities and empty removed

In [14]:
train_loss, valid_loss = util.train(model, train_loader, test_loader, criterion, optimizer, device, patience=4, num_epochs=20) # with entities 3%

Epoch [1/20] - Training Loss: 2.1146- Validation Loss: 1.3011
Epoch [2/20] - Training Loss: 1.0889- Validation Loss: 1.0722
Epoch [3/20] - Training Loss: 0.7658- Validation Loss: 1.0116
Epoch [4/20] - Training Loss: 0.5485- Validation Loss: 1.0325
Epoch [5/20] - Training Loss: 0.3898- Validation Loss: 1.0706
Epoch [6/20] - Training Loss: 0.2780- Validation Loss: 1.1378
Epoch [7/20] - Training Loss: 0.2065- Validation Loss: 1.1919
Early stopping after 7 epochs without improvement.


In [17]:
evaluate_model(model, test_loader,device) # with entities All

Accuracy: 0.7156
Precision: 0.7165
Recall: 0.7134
F1 Score: 0.7118


In [20]:
train_loss, valid_loss = util.train(model, train_loader, test_loader, criterion, optimizer, device, patience=4, num_epochs=20) # with entities All

Epoch [1/20] - Training Loss: 2.0879- Validation Loss: 1.3162
Epoch [2/20] - Training Loss: 1.0802- Validation Loss: 1.0909
Epoch [3/20] - Training Loss: 0.7499- Validation Loss: 1.0562
Epoch [4/20] - Training Loss: 0.5295- Validation Loss: 1.0380
Epoch [5/20] - Training Loss: 0.3737- Validation Loss: 1.0654
Epoch [6/20] - Training Loss: 0.2687- Validation Loss: 1.1337
Epoch [7/20] - Training Loss: 0.1962- Validation Loss: 1.2125
Epoch [8/20] - Training Loss: 0.1493- Validation Loss: 1.2774
Early stopping after 8 epochs without improvement.


In [22]:
evaluate_model(model, test_loader,device) # with entities All

Accuracy: 0.7188
Precision: 0.7166
Recall: 0.7173
F1 Score: 0.7124


In [14]:
def evaluate_model(model, data_loader,device):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_predictions = []
    all_targets = []
 
    with torch.no_grad():
        for input_ids, attention_mask, targets in data_loader:
            input_ids, attention_mask, targets = input_ids.to(device), attention_mask.to(device), targets.to(device)
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == targets).sum().item()
            total_samples += targets.size(0)
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    average_loss = total_loss / len(data_loader)

    accuracy = accuracy_score(all_targets, all_predictions)
    precision = precision_score(all_targets, all_predictions, average='macro')
    recall = recall_score(all_targets, all_predictions, average='macro')
    f1 = f1_score(all_targets, all_predictions, average='macro')
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import json
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import spacy
import Utility as util
from Utility import REDataset_entities, REModelWithAttention

In [2]:
df_fewRel = pd.read_csv('fewRel_entities.csv')

In [3]:
df_aug_llama = pd.read_csv('aug_llama_entities_cleaned.csv')

In [4]:
print(f'orignal dataset = {len(df_fewRel)} and augumented data = {len(df_aug_llama)}')

orignal dataset = 44800 and augumented data = 2783


In [6]:
df_aug_llama = util.sampleData(df_aug_llama,'relation')

In [5]:
len(df_aug_llama)

2783

In [6]:
train_df, test_df = train_test_split(df_fewRel, test_size=0.2, random_state=42) # To take test set from orignal data
print(f'Train set = {len(train_df)} and test set = {len(test_df)}')

Train set = 35840 and test set = 8960


In [7]:
df_combined = pd.concat([train_df, df_aug_llama], ignore_index=True) # combined augumented data with orignal FewRel training data
print(len(df_combined))

38623


In [8]:
# Encode training and test set for relation
label_encoder = LabelEncoder()
df_combined['relation_id'] = label_encoder.fit_transform(df_combined['relation'])
test_df['relation_id'] = label_encoder.fit_transform(test_df['relation'])

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # Choose the same model as in your model
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [10]:
max_seq_length = 128  
train_dataset = util.REDataset_entities(df_combined,tokenizer, max_seq_length)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = util.REDataset_entities(test_df, tokenizer, max_seq_length)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [11]:
model = REModelWithAttention(tokenizer, num_classes=len(label_encoder.classes_)).to(device)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [ ]:
train_loss, valid_loss = util.train(model, train_loader, test_loader, criterion, optimizer, device, patience=4, num_epochs=20) # All with entities also empty removed

Epoch [1/20] - Training Loss: 2.1503- Validation Loss: 1.3061
Epoch [2/20] - Training Loss: 1.0838- Validation Loss: 1.0952
Epoch [3/20] - Training Loss: 0.7501- Validation Loss: 1.0367
Epoch [4/20] - Training Loss: 0.5297- Validation Loss: 1.0666
Epoch [5/20] - Training Loss: 0.3756- Validation Loss: 1.1117
Epoch [6/20] - Training Loss: 0.2692- Validation Loss: 1.1699
Epoch [7/20] - Training Loss: 0.1919- Validation Loss: 1.2417
Early stopping after 7 epochs without improvement.


In [ ]:
evaluate_model(model, test_loader,device) # All with entities also empty removed

Accuracy: 0.7100
Precision: 0.7153
Recall: 0.7082
F1 Score: 0.7086


In [14]:
train_loss, valid_loss = util.train(model, train_loader, test_loader, criterion, optimizer, device, patience=4, num_epochs=20) # with entities 3%

Epoch [1/20] - Training Loss: 2.1146- Validation Loss: 1.3011
Epoch [2/20] - Training Loss: 1.0889- Validation Loss: 1.0722
Epoch [3/20] - Training Loss: 0.7658- Validation Loss: 1.0116
Epoch [4/20] - Training Loss: 0.5485- Validation Loss: 1.0325
Epoch [5/20] - Training Loss: 0.3898- Validation Loss: 1.0706
Epoch [6/20] - Training Loss: 0.2780- Validation Loss: 1.1378
Epoch [7/20] - Training Loss: 0.2065- Validation Loss: 1.1919
Early stopping after 7 epochs without improvement.


In [17]:
evaluate_model(model, test_loader,device) #  with entities 3%

Accuracy: 0.7156
Precision: 0.7165
Recall: 0.7134
F1 Score: 0.7118


In [20]:
train_loss, valid_loss = util.train(model, train_loader, test_loader, criterion, optimizer, device, patience=4, num_epochs=20) # with entities All

Epoch [1/20] - Training Loss: 2.0879- Validation Loss: 1.3162
Epoch [2/20] - Training Loss: 1.0802- Validation Loss: 1.0909
Epoch [3/20] - Training Loss: 0.7499- Validation Loss: 1.0562
Epoch [4/20] - Training Loss: 0.5295- Validation Loss: 1.0380
Epoch [5/20] - Training Loss: 0.3737- Validation Loss: 1.0654
Epoch [6/20] - Training Loss: 0.2687- Validation Loss: 1.1337
Epoch [7/20] - Training Loss: 0.1962- Validation Loss: 1.2125
Epoch [8/20] - Training Loss: 0.1493- Validation Loss: 1.2774
Early stopping after 8 epochs without improvement.


In [22]:
evaluate_model(model, test_loader,device) # with entities All

Accuracy: 0.7188
Precision: 0.7166
Recall: 0.7173
F1 Score: 0.7124


In [16]:
def evaluate_model(model, data_loader,device):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_predictions = []
    all_targets = []
 
    with torch.no_grad():
        for input_ids, attention_mask, targets in data_loader:
            input_ids, attention_mask, targets = input_ids.to(device), attention_mask.to(device), targets.to(device)
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == targets).sum().item()
            total_samples += targets.size(0)
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    average_loss = total_loss / len(data_loader)

    accuracy = accuracy_score(all_targets, all_predictions)
    precision = precision_score(all_targets, all_predictions, average='macro')
    recall = recall_score(all_targets, all_predictions, average='macro')
    f1 = f1_score(all_targets, all_predictions, average='macro')
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")